## Exploratory Data Analysis

Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA.


We shouldn't limit our EDA to these 10 questions. Let's be creative :).

In [20]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2

In [ ]:
df = pd.read_csv('flights.csv', nrows=10000)

In [ ]:
#df1, df2, df3, df4 = np.array_split(df, 4)

In [54]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)

    return table


test_qu = "SELECT * From flights WHERE arr_delay IS NOT NULL AND arr_delay < 200 limit 100000"

test_pd_table = create_pandas_table(test_qu)

# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

test_pd_table.head() # show table querried

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-07-20,AA,AA_CODESHARE,AA,3446,MQ,N662EH,3446,11996,GSP,...,637.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-07-20,AA,AA_CODESHARE,AA,3448,MQ,N835AE,3448,12511,JLN,...,484.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-07-20,AA,AA_CODESHARE,AA,3449,MQ,N809AE,3449,11298,DFW,...,692.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-07-20,AA,AA_CODESHARE,AA,3450,MQ,N835AE,3450,13930,ORD,...,484.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-07-20,AA,AA_CODESHARE,AA,3452,MQ,N232NN,3452,15919,XNA,...,280.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [55]:
df = test_pd_table

In [56]:
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [57]:
df.shape

(100000, 42)

#### **Task 1**: Test the hypothesis that the delay is from Normal distribution. and that **mean** of the delay is 0. Be careful about the outliers.

In [58]:
df['arr_delay'].agg(['max', 'min', 'mean', 'std', 'median'])

max       199.000000
min       -70.000000
mean        7.458160
std        36.873741
median     -4.000000
Name: arr_delay, dtype: float64

In [48]:
df['arr_delay'].sort_values()

4980      -87.0
15582     -63.0
18912     -62.0
2568      -61.0
29843     -60.0
          ...  
1247     1262.0
33568    1293.0
36064    1316.0
44064    1385.0
86811    1747.0
Name: arr_delay, Length: 100000, dtype: float64

In [53]:
len(df[df['arr_delay'] > 200])

1448

In [59]:
stat, p = shapiro(df.arr_delay)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

stat=0.738, p=0.000
Probably not Gaussian


/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [60]:
tset, pval = ttest_1samp(df.arr_delay, 0)

In [61]:
print("p-values",pval)
if pval < 0.05:    # alpha value is 0.05 or 5%
   print(" we are rejecting null hypothesis")
else:
  print("we are accepting null hypothesis")

p-values 0.0
 we are rejecting null hypothesis


#### **Task 2**: Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**: What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

#### **Task 6**: How many states cover 50% of US air traffic? 

#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?